In [1]:
print(5)

5


In [2]:
import requests
from bs4 import BeautifulSoup
import json  # Import the JSON module
categories_to_search = ['beauty','womens+bags',
                        'fragrances','skin+care',
                        'sunglasses','womens+dresses',
                        'womens+jewellery',
                        'womens+shoes','womens+watches']
products = []
item = 1
for category in categories_to_search :
    
    # Amazon URL for beauty products
    URL = f"https://www.amazon.com/s?k={category}+products"
    
    # Headers to mimic a browser request
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }

    # Send the request
    response = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract product details
    
    for product in soup.find_all("div", {"data-component-type": "s-search-result"}):
        try:
            # Title
            title = product.h2.text.strip() if product.h2 else "N/A"
            
            # Description (Amazon often has limited inline description)
            description_tag = product.find("img", class_="s-image")
            description = description_tag["alt"] if description_tag else "N/A"
            
            # Price
            price_whole = product.find("span", class_="a-price-whole")
            price_fraction = product.find("span", class_="a-price-fraction")
            if price_whole and price_fraction:
                price = f"{price_whole.text.strip()}.{price_fraction.text.strip()}"
            else:
                price = "N/A"
            
            # Rating
            rating_tag = product.find("span", class_="a-icon-alt")
            rating = rating_tag.text.strip() if rating_tag else "N/A"
            
            # Stock (Amazon often doesn't display this on the search results)
            stock = "Available" if product.find("span", {"class": "a-color-price"}) else "UnAvailable"
            
            # Image
            image_tag = product.find("img", class_="s-image")
            image_url = image_tag["src"] if image_tag else "N/A"
            
            # Append to the products list
            products.append({
                "id":item,
                "title": title,
                "description": description,
                "category":category,
                "price": price,
                "rating": rating,
                "stock": stock,
                "image": image_url,
                "reviews":[]
            })
            item = item + 1
        except Exception as e:
            print(f"Error extracting product: {e}")
    
    # Save to a JSON file
with open("products.json", "w", encoding="utf-8") as json_file:
    json.dump(products, json_file, ensure_ascii=False, indent=4)

print("Scraping complete. Data saved to products.json")


Scraping complete. Data saved to products.json
